Script para estudar parelalização e depois aplicar no código de Simulated Annealing

Referências:

1. https://discourse.julialang.org/t/questions-about-getting-started-with-parallel-computing/25341

2. https://github.com/crstnbr/JuliaWorkshop18/blob/master/5%20Parallel%20computing/parallel-computing.ipynb

In [2]:
using BenchmarkTools
using FLoops
using Distributed


Distributed

In [3]:
nprocs()

1

In [4]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [5]:
nprocs() #número de cores. O primeiro core é sempre o mestre

5

In [6]:
nworkers() #número de cores trabalhadores

4

A macro `@everywhere` faz os pacotes, funções ou dados serem distribuídos a todos os processadores, em todos os lugares

In [9]:
@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

In [10]:
#Vh/D, L/D, Dj/D
@everywhere data = [1.0, 0.25, 0.68]; 

#ρ, β, c0, λj
@everywhere game_parameters = [0.884, 9.84, 0.044, 0.346];

In [13]:
@everywhere function solve_tree_serial(n, data=data, game_parameters=game_parameters)
    for i in 1:n
        solve_tree(data, game_parameters)
        
    end
end

In [14]:
@time solve_tree_serial(1)

339.344697 seconds (635.94 M allocations: 210.131 GiB, 8.98% gc time)
340.972184 seconds (638.98 M allocations: 210.524 GiB, 9.00% gc time)


In [15]:
?pmap()

```
pmap(f, [::AbstractWorkerPool], c...; distributed=true, batch_size=1, on_error=nothing, retry_delays=[], retry_check=nothing) -> collection
```

Transform collection `c` by applying `f` to each element using available workers and tasks.

For multiple collection arguments, apply `f` elementwise.

Note that `f` must be made available to all worker processes; see [Code Availability and Loading Packages](@ref code-availability) for details.

If a worker pool is not specified, all available workers, i.e., the default worker pool is used.

By default, `pmap` distributes the computation over all specified workers. To use only the local process and distribute over tasks, specify `distributed=false`. This is equivalent to using [`asyncmap`](@ref). For example, `pmap(f, c; distributed=false)` is equivalent to `asyncmap(f,c; ntasks=()->nworkers())`

`pmap` can also use a mix of processes and tasks via the `batch_size` argument. For batch sizes greater than 1, the collection is processed in multiple batches, each of length `batch_size` or less. A batch is sent as a single request to a free worker, where a local [`asyncmap`](@ref) processes elements from the batch using multiple concurrent tasks.

Any error stops `pmap` from processing the remainder of the collection. To override this behavior you can specify an error handling function via argument `on_error` which takes in a single argument, i.e., the exception. The function can stop the processing by rethrowing the error, or, to continue, return any value which is then returned inline with the results to the caller.

Consider the following two examples. The first one returns the exception object inline, the second a 0 in place of any exception:

```julia-repl
julia> pmap(x->iseven(x) ? error("foo") : x, 1:4; on_error=identity)
4-element Array{Any,1}:
 1
  ErrorException("foo")
 3
  ErrorException("foo")

julia> pmap(x->iseven(x) ? error("foo") : x, 1:4; on_error=ex->0)
4-element Array{Int64,1}:
 1
 0
 3
 0
```

Errors can also be handled by retrying failed computations. Keyword arguments `retry_delays` and `retry_check` are passed through to [`retry`](@ref) as keyword arguments `delays` and `check` respectively. If batching is specified, and an entire batch fails, all items in the batch are retried.

Note that if both `on_error` and `retry_delays` are specified, the `on_error` hook is called before retrying. If `on_error` does not throw (or rethrow) an exception, the element will not be retried.

Example: On errors, retry `f` on an element a maximum of 3 times without any delay between retries.

```julia
pmap(f, c; retry_delays = zeros(3))
```

Example: Retry `f` only if the exception is not of type [`InexactError`](@ref), with exponentially increasing delays up to 3 times. Return a `NaN` in place for all `InexactError` occurrences.

```julia
pmap(f, c; on_error = e->(isa(e, InexactError) ? NaN : rethrow()), retry_delays = ExponentialBackOff(n = 3))
```


In [16]:
function solve_tree_parallel(n_array)
    pmap(solve_tree_serial, n_array)
end


solve_tree_parallel (generic function with 1 method)

In [17]:
@time solve_tree_parallel([1,1])

      From worker 5:	404.974531 seconds (635.94 M allocations: 210.131 GiB, 8.08% gc time)
      From worker 2:	405.674996 seconds (635.94 M allocations: 210.131 GiB, 8.13% gc time)
408.413957 seconds (389.69 k allocations: 20.075 MiB)


2-element Array{Nothing,1}:
 nothing
 nothing

In [18]:
[1,1]

2-element Array{Int64,1}:
 1
 1

In [21]:
ones(nworkers(), Int6)

LoadError: MethodError: no method matching ones(::Int64, ::Type{Int64})
Closest candidates are:
  ones(!Matched::Union{Integer, AbstractUnitRange}...) at array.jl:520

In [28]:
array_workers = ones(Int64, nworkers());

@time solve_tree_parallel(array_workers)

      From worker 3:	678.712641 seconds (635.94 M allocations: 210.131 GiB, 7.00% gc time)
      From worker 4:	687.703812 seconds (635.94 M allocations: 210.131 GiB, 6.73% gc time)
      From worker 5:	708.036772 seconds (632.36 M allocations: 209.969 GiB, 7.90% gc time)
      From worker 2:	709.146085 seconds (632.36 M allocations: 209.969 GiB, 7.88% gc time)
709.404949 seconds (19.64 k allocations: 595.422 KiB)


4-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing